# Load LoRA adapters

In [3]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
    raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

OK


In [2]:
%pip install bitsandbytes==0.37.0 transformers==4.27.4 datasets==2.7.0 accelerate==0.18.0 loralib==0.1.1 peft==0.3.0.dev0

Note: you may need to restart the kernel to use updated packages.


#### Load base model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

CACHE_DIR = '/media/tfsservices/DATA/NLP/cache/'
MODEL_NAME = "facebook/opt-6.7b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,        # bitsandbytes lib required (convert the loaded model into mixed-8bit quantized model.)
    device_map='auto',
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR)      # path to a directory in which a downloaded pretrained model
    # low_cpu_mem_usage=True,   # loads the model using ~1x model size CPU memory
    # offload_state_dict=True)  # temporarily offload the CPU state dict to the hard drive to avoid getting out of CPU RAM
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]


In [7]:
def test_generate_marketing():

    inputs = tokenizer("Hi {FirstName}", return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=150)
        print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [9]:
model.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear8bitLt(in_features=16384, out_features=4096, bias=True

In [8]:
test_generate_marketing()

Hi {FirstName}!

We’re excited to share that you’ve been selected to receive a free copy of the new book, The Art of the Deal: The Art of Negotiation in the Real World, by President Donald J. Trump.

The Art of the Deal is a classic that has sold more than 30 million copies worldwide. It’s a must-read for anyone who wants to learn how to negotiate and succeed in business.

The Art of the Deal is a classic that has sold more than 30 million copies worldwide. It’s a must-read for anyone who wants to learn how to negotiate and succeed in business.

The Art of the Deal is a classic that has sold more than 30


#### Load and apply LoRA adapter

In [12]:
from peft import PeftModel, PeftConfig

peft_model_id = "lora_adapters"

config = PeftConfig.from_pretrained(peft_model_id)
model = PeftModel.from_pretrained(model, peft_model_id)


In [13]:
test_generate_marketing()

Hi {FirstName} -

I am working with a client who is releasing {MakeModel} {Category} {Qty} units. They are asking only {Price} for all {Qty} units.

The seller is only looking for quick sale, so they can get the money back asap.

Please review the photos and let me know if you are interested.

{Listing}

{SendersSignature}

{SendersSignature}

{Listing}

{Listing}

{Listing}

{Listing}

{Listing}

{Listing}

{Listing}

{Listing}

{List
